In [ ]:
# !pip install ISLP

In [ ]:
#basic imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#preprossing imports
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#model development imports
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn import tree
from sklearn.naive_bayes import MultinomialNB as MNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from ISLP import confusion_table as ct

In [ ]:
data = pd.read_excel('/content/sample_data/Dry_Bean_Dataset.xlsx')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
null_values = data.isnull().sum()
null_values

In [ ]:
data = data.dropna()
data.shape

In [ ]:
# data.info()
data.describe()

In [ ]:
#Preprossing steps

In [ ]:
#replacing class data with values
set(data['Class'])
encoder = LabelEncoder()
data['Class'] = encoder.fit_transform(data['Class'])
data.head()

In [ ]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

In [ ]:
# tried mim max scaler as well but not able to scale well since variance is very high
scaler = RobustScaler()
features = data.columns.drop(['Class' , 'Solidity' , 'ShapeFactor2' , 'ShapeFactor4' , 'ShapeFactor3'])
data[features] = scaler.fit_transform(data[features])

# to make data +ve
for col in data:
    if data[col].min() < 0:
        data[col] += abs(data[col].min())

data.head()

In [ ]:
data.describe()

In [ ]:
plt.scatter(data['Area'] ,data['Class'] )

In [ ]:
data.columns

In [ ]:
# selected_features = ['Extent',
#        'Solidity', 'roundness', 'Compactness', 'ShapeFactor1','Class']
# sub_Data = data[selected_features]
# type(sub_Data)
# sns.pairplot(sub_Data, hue='Class', kind='scatter') # very cost heavy process
# data


# df_long = data.melt(id_vars=['Class'], var_name='Feature')
# df_long
# Plot categorical scatter plot of each feature against 'Class'
# sns.catplot(data=df_long, x='Class', y='value', hue='Feature', kind='swarm') # cost heavy operation


# pred = data[data.columns.drop(['Class'])]
# label = data['Class']
# type(pred) # may be i should do after some steps

In [ ]:
#outliers
orf = ['ShapeFactor2','MajorAxisLength', 'Eccentricity','Solidity'] # outliers_removal_features , i plotted scatter to see weather outliers are present or not
z_scores = (data[orf] - data[orf].mean()) / data[orf].std()
limiter = 5
data = data[(z_scores.abs() < limiter).all(axis=1)]
data.describe()

In [ ]:
data.head()

In [ ]:
# plt.scatter(data['Eccentricity'] ,data['Class'] )

In [ ]:
features = data.columns[:-1]

for feature in features:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=feature, y='Class', data=data, alpha=0.5)
    # sns.lineplot(x=feature, y='Class', data=data.groupby(feature)['Class'].mean(), color='red')
    plt.title(f'Scatter and Line Plot for {feature} vs. Class')
    plt.xlabel(feature)
    plt.ylabel('Class')
    plt.grid(True)



In [ ]:
# sns.pairplot(data, hue='Class', diag_kind='kde') # very cost heavy process

In [ ]:
data.head(1)

In [ ]:
# bad way to edit data
# names=['AR/ECC', 'AR/PE/MA/ED/CA', 'CO/SF3']
# features = ['AspectRation', 'Eccentricity', 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea', 'Compactness', 'ShapeFactor3']


# combined_feature = data[['AspectRation' ,'Eccentricity' ]].mean(axis = 1)
# data['Ar/Ecc'] = combined_feature

# combined_feature2 = data[[ 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea' ]].mean(axis = 1)
# data['AR/PE/MA/ED/CA'] = combined_feature2

# combined_feature3 = data[['Compactness', 'ShapeFactor3' ]].mean(axis = 1)
# data['CO/SF3'] = combined_feature3

# data.drop(['AspectRation', 'Eccentricity', 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea', 'Compactness', 'ShapeFactor3'] , axis=1,inplace=True)
# data.head()


# bad way to modify

# # Define the combine_features function
# def combine_features(X):
#     combined_features = []
#     combined_features.append(X[['AspectRation', 'Eccentricity']].mean(axis=1))
#     combined_features.append(X[['Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea']].mean(axis=1))
#     combined_features.append(X[['Compactness', 'ShapeFactor3']].mean(axis=1))
#     return pd.concat(combined_features, axis=1)

# names=['AR/ECC', 'AR/PE/MA/ED/CA', 'CO/SF3']

# # Define the combined_feature_transformer
# combined_feature_transformer = ColumnTransformer(
#     transformers=[
#         ('combine', FunctionTransformer(validate=False), ['AspectRation', 'Eccentricity', 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea', 'Compactness', 'ShapeFactor3'])
#     ],
#     remainder='passthrough'
# )

# # Define the pipeline
# pipeline = Pipeline([
#     ('combine_features', combined_feature_transformer)
# ])

# # Fit and transform the pipeline
# combined_features = pipeline.fit_transform(df)


# df_combined = pd.DataFrame(combined_features, index=df.index, names=['AR/ECC', 'AR/PE/MA/ED/CA', 'CO/SF3'])
# print(df_combined)


In [ ]:
# data2 = data.copy()
features = ['AspectRation', 'Eccentricity', 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea', 'Compactness', 'ShapeFactor3']
names = ['AR/ECC', 'AR/PE/MA/ED/CA', 'CO/SF3']


combined_features_df = pd.DataFrame()


for name, feature_group in zip(names, [features[:2], features[2:7], features[7:]]):
    combined_feature = data[feature_group].mean(axis=1)
    combined_features_df[name] = combined_feature

combined_features_df

data = pd.concat([data , pd.DataFrame(combined_features_df)] , axis = 1 )
data.drop(['AspectRation', 'Eccentricity', 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea', 'Compactness', 'ShapeFactor3'] , axis=1,inplace=True)
data.head()

In [ ]:
#to shift Y(class) to right
col = data.pop('Class')
data['Class'] = col
# data.head(1)
data.describe()

##<------------------------------ Preprocessing Part Done ----------------------------->


Certainly! Here's the breakdown:

## Using scikit-learn (sklearn):

Supervised methods:
- Logistic Regression
- KNN (K-Nearest Neighbors)
- LDA (Linear Discriminant Analysis)
- QDA (Quadratic Discriminant Analysis)
- Decision Trees
- Naive Bayes

Unsupervised methods:
- K-Means Clustering
- Hierarchical Clustering

## Using TensorFlow (tf):

Supervised methods:

- SVM (Support Vector Machines)
- Neural Network

In [ ]:
#calculating no of features
for i in range(7):
  print(i ,'count :' ,(data['Class'] == i).sum() )

In [ ]:
X = data[data.columns.drop('Class')]
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Logistic Regression

In [ ]:
#100 iteration gives same accuracy as 1lakh iterations
logistic_reg = LogisticRegression(max_iter=100, multi_class='multinomial', solver='lbfgs')


logistic_reg.fit(X_train, y_train)
y_pred = logistic_reg.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
# print(conf_matrix)
ct(y_pred,y_test) # islp package shows good confusion table

KNN

In [ ]:
no_of_neigh = 15 # no of neighbours
knn = KNeighborsClassifier(n_neighbors=no_of_neigh)
knn.fit(X_train , y_train)
y_pred_knn = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("Classification Report:")
print(classification_report(y_test, y_pred_knn))

ct(y_test , y_pred_knn)

Linear Discriminant Analysis

In [ ]:
clf = LDA()
clf.fit(X_train, y_train)
y_pred_lda = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_lda))
print("Classification Report:")
print(classification_report(y_test, y_pred_lda))

ct(y_test , y_pred_lda)

In [ ]:
qlf = QDA()
qlf.fit(X_train, y_train)
y_pred_qda = qlf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_qda))
print("Classification Report:")
print(classification_report(y_test, y_pred_qda))

ct(y_test , y_pred_qda)

Decision Trees

In [ ]:
decision_tree = tree.DecisionTreeClassifier()
decision_tree.fit(X_train , y_train)
y_pred_decTree = decision_tree.predict(X_test)


print("Accuracy:", accuracy_score(y_test, y_pred_decTree))
print("Classification Report:")
print(classification_report(y_test, y_pred_decTree))

ct(y_test , y_pred_decTree)

Naive Bayes

In [ ]:
naive_model = MNB()
naive_model.fit(X_train,y_train)
y_pred_nb = naive_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print("Classification Report:")
print(classification_report(y_test, y_pred_nb))

ct(y_test , y_pred_nb)